https://blog.csdn.net/gavin3318/article/details/78499419 
Python 中的串行执行 并发执行 同步异步

程序的执行:

串行执行:ffd

In [17]:
import requests
import time

def get_page(url):
    print('GET: %s' %url)
    response=requests.get(url)
    print(url,len(response.text))
    return response.text

urls=[
    'https://www.baidu.com',
    'https://www.python.org',
    'https://www.openstack.org'
]

#程序的执行是串行执行
start=time.time()
for url in urls:
   res=get_page(url) #任务的调用方式是同步调用：提交一个任务，然后在原地等待，等待返回结果后再执行下一行代码
stop=time.time()
print(stop-start) #2.664152145385742

GET: https://www.baidu.com
https://www.baidu.com 2443
GET: https://www.python.org
https://www.python.org 49122
GET: https://www.openstack.org
https://www.openstack.org 37707
1.666886806488037


并发执行(并行执行):

In [18]:
from multiprocessing import Process
from threading import Thread
import requests
import time

def get_page(url):
    print('GET: %s' %url)
    response=requests.get(url)
    print(url,len(response.text))
    return response.text

urls=[
    'https://www.baidu.com',
    'https://www.python.org',
    'https://www.openstack.org'
]

if __name__ == '__main__':#在python中开启多进程,需要在__main__下执行

    #程序的执行是串行执行
    start=time.time()
    p_l=[]
    for url in urls:
       p=Thread(target=get_page,args=(url,))#多线程
       p_l.append(p)
       # p = Process(target=get_page, args=(url,)) #多进程
       # p_l.append(p)
       p.start()
    for p in p_l:
        p.join()
    stop=time.time()
    print(stop-start) #2.664152145385742

GET: https://www.baidu.comGET: https://www.python.org

GET: https://www.openstack.org
https://www.baidu.com 2443
https://www.python.org 49122
https://www.openstack.org 37832
0.926692008972168


进程池 线程池:

In [24]:
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor

from multiprocessing import Process
from threading import Thread
import requests
import time

def get_page(url):
    print('GET: %s' %url)
    response=requests.get(url)
    return len(response.text)

urls=[
    'https://www.baidu.com',
    'https://www.python.org',
    'https://www.openstack.org',
    'https://www.guorn.com'
]

if __name__ == '__main__':
    p=ProcessPoolExecutor(1)#默认是你CPU核心数量

    #程序的执行是串行执行
    start=time.time()
    l=[]
    for url in urls:
        furture=p.submit(get_page,url) #任务的调用方式是异步调用：提交一个任务，不用等待，直接执行下一行代码
        l.append(furture)
    p.shutdown(wait=True)
    for f in l:
        print(f.result())
    stop=time.time()
    print(stop-start) #2.664152145385742

Exception ignored in: <function ProcessPoolExecutor._start_queue_management_thread.<locals>.weakref_cb at 0x000001B7CEB6EEE8>
Traceback (most recent call last):
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\concurrent\futures\process.py", line 581, in weakref_cb
    thread_wakeup.wakeup()
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\concurrent\futures\process.py", line 90, in wakeup
    self._writer.send_bytes(b"")
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\multiprocessing\connection.py", line 183, in send_bytes
    self._check_closed()
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\multiprocessing\connection.py", line 136, in _check_closed
    raise OSError("handle is closed")
OSError: handle is closed


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

异步调用时候要结合回调机制来使用.


In [27]:
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor

from multiprocessing import Process
from threading import Thread
import requests
import time,os,random

def get_page(url):
    print('%s GET: %s' %(os.getpid(),url))
    response=requests.get(url)
    # time.sleep(random.randint(1,3))
    return len(response.text)

def parse_page(res):
    res=res.result()
    print('%s parse res: %s' %(os.getpid(),res))

urls=[
    'https://www.baidu.com',
    'https://www.python.org',
    'https://www.openstack.org',
    'https://www.guorn.com'
]

if __name__ == '__main__':
    p=ProcessPoolExecutor(1)

    #程序的执行是串行执行
    start=time.time()
    for url in urls:
        p.submit(get_page,url).add_done_callback(parse_page)
        p.submit(get_page,url)
    p.shutdown(wait=True)
    stop=time.time()
    print('执行时间：',stop-start) #2.664152145385742

    print('主',os.getpid())

Exception ignored in: <function ProcessPoolExecutor._start_queue_management_thread.<locals>.weakref_cb at 0x000001B7CED2C708>
Traceback (most recent call last):
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\concurrent\futures\process.py", line 581, in weakref_cb
    thread_wakeup.wakeup()
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\concurrent\futures\process.py", line 90, in wakeup
    self._writer.send_bytes(b"")
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\multiprocessing\connection.py", line 183, in send_bytes
    self._check_closed()
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\multiprocessing\connection.py", line 136, in _check_closed
    raise OSError("handle is closed")
OSError: handle is closed
exception calling callback for <Future at 0x1b7cf378388 state=finished raised BrokenProcessPool>
Traceback (most recent call last):
  File "C:\Users\Aaron\AppData\Local\Programs\Python\Python37\lib\concur

In [25]:
import aiohttp
import threading
async def get(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            print(url, resp.status)
            print('Hello world!  thread=%s' % ( threading.currentThread()))
            # print(url, await resp.text())

#程序的执行是串行执行
start=time.time()
loop = asyncio.get_event_loop()     # 得到一个事件循环模型
tasks = [                           # 初始化任务列表
    get("http://zhushou.360.cn/detail/index/soft_id/3283370"),
    get("http://zhushou.360.cn/detail/index/soft_id/3264775"),
    get("http://zhushou.360.cn/detail/index/soft_id/705490")
]
loop.run_until_complete(asyncio.wait(tasks))    # 执行任务

stop=time.time()
print('执行时间：',stop-start) #2.664152145385742
# loop.close()

http://zhushou.360.cn/detail/index/soft_id/3264775 200
Hello world!  thread=<_MainThread(MainThread, started 17972)>
http://zhushou.360.cn/detail/index/soft_id/705490 200
Hello world!  thread=<_MainThread(MainThread, started 17972)>
http://zhushou.360.cn/detail/index/soft_id/3283370 200
Hello world!  thread=<_MainThread(MainThread, started 17972)>
执行时间： 0.05899238586425781


In [8]:
# 范例 https://zhuanlan.zhihu.com/p/24118476?utm_source=cn.ticktick.task&utm_medium=social&utm_oi=51474960744448
import asyncio
import threading
import nest_asyncio
nest_asyncio.apply()

# 异步IO例子：适配Python3.5，使用async和await关键字
async def hello(index):       # 通过关键字async定义协程
    print('Hello world! index=%s, thread=%s' % (index, threading.currentThread()))
    await asyncio.sleep(1)     # 模拟IO任务
    print('Hello again! index=%s, thread=%s' % (index, threading.currentThread()))

loop = asyncio.get_event_loop()     # 得到一个事件循环模型
tasks = [hello(1), hello(2)]        # 初始化任务列表
loop.run_until_complete(asyncio.wait(tasks))    # 执行任务
# loop.close() 

({<Task finished coro=<hello() done, defined at <ipython-input-8-046c8b62146b>:6> exception=NameError("name 'threading' is not defined")>,
  <Task finished coro=<hello() done, defined at <ipython-input-8-046c8b62146b>:6> exception=NameError("name 'threading' is not defined")>},
 set())

In [32]:
# https://zhuanlan.zhihu.com/p/34324225 python异步asyncio模块的使用

# 第一种用法

# 顺序执行
def myfun_seq(i):
    time.sleep(1)

start=time.time()
# myfun_seq_list = (myfun_seq(i) for i in range(10))
for i in range(10):
    myfun_seq(i)
    print(i)

stop=time.time()
print('执行时间：',stop-start)


import asyncio

async def myfun(i):
    print('start {}th'.format(i))
    await asyncio.sleep(1)
    print('finish {}th'.format(i))

# 这个例子很好，i可以换成密码
start=time.time()
loop = asyncio.get_event_loop()
myfun_list = (myfun(i) for i in range(10))
loop.run_until_complete(asyncio.gather(*myfun_list))
stop=time.time()
print('执行时间：',stop-start) #2.664152145385742

执行时间： 10.005696058273315
start 0th
start 1th
start 2th
start 3th
start 4th
start 5th
start 6th
start 7th
start 8th
start 9th
finish 0th
finish 2th
finish 6th
finish 9th
finish 8th
finish 5th
finish 7th
finish 4th
finish 1th
finish 3th
执行时间： 1.007601261138916


In [37]:
# 第二种用法
import asyncio

async def myfun(i):
    print('start {}th'.format(i))
    await asyncio.sleep(1)
    # time.sleep(1) !!!!!!!!!!!!这个函数在这里无法实现异步，只能通过添加线程实现
    print('finish {}th'.format(i))
start=time.time()
loop = asyncio.get_event_loop()
myfun_list = [asyncio.ensure_future(myfun(i)) for i in range(10)]
loop.run_until_complete(asyncio.wait(myfun_list))
stop=time.time()
print('执行时间：',stop-start) #2.664152145385742

start 0th
start 1th
start 2th
start 3th
start 4th
start 5th
start 6th
start 7th
start 8th
start 9th
finish 0th
finish 2th
finish 6th
finish 9th
finish 8th
finish 5th
finish 7th
finish 4th
finish 1th
finish 3th
执行时间： 0.9971179962158203


In [33]:
# 一般的函数无法实现异步

import asyncio
import requests
from bs4 import BeautifulSoup

async def get_title(a):
    url = 'https://movie.douban.com/top250?start={}&filter='.format(a*25)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    lis = soup.find('ol', class_='grid_view').find_all('li')
    for li in lis:
        title = li.find('span', class_="title").text
        print(title)

loop = asyncio.get_event_loop()
fun_list = (get_title(i) for i in range(10))
loop.run_until_complete(asyncio.gather(*fun_list))

AttributeError: 'NoneType' object has no attribute 'find_all'

In [35]:
# 对于上述函数，asyncio库只能通过添加线程的方式实现异步，下面我们实现time.sleep时的异步

import asyncio
import time

def myfun(i):
    print('start {}th'.format(i))
    time.sleep(1)
    print('finish {}th'.format(i))

async def main():
    loop = asyncio.get_event_loop()
    futures = (
        loop.run_in_executor(
            None,
            myfun, 
            i)
        for i in range(10)
        )
    for result in await asyncio.gather(*futures):
        pass

start=time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

stop=time.time()
print('执行时间：',stop-start) #2.664152145385742

start 0th
start 1th
start 2th
start 3th
start 4th
start 5th
start 6thstart 7th

start 8th
start 9th
finish 0th
finish 1th
finish 2th
finish 3th
finish 4th
finish 5th
finish 7thfinish 6th

finish 9thfinish 8th

执行时间： 1.0554842948913574


In [40]:
# 上面10次循环仍然不是一次性打印出来的，而是像分批次一样打印出来的。这是因为开启的线程不够多，如果想要实现一次打印，可以开启10个线程，代码如下

import concurrent.futures as cf # 多加一个模块
import asyncio
import time

def myfun(i):
    print('start {}th'.format(i))
    time.sleep(1)
    print('finish {}th'.format(i))

async def main():
    with cf.ThreadPoolExecutor(max_workers = 10) as executor: # 设置10个线程
        loop = asyncio.get_event_loop()
        futures = (
            loop.run_in_executor(
                executor, # 按照10个线程来执行
                myfun, 
                i)
            for i in range(10)
            )
        for result in await asyncio.gather(*futures):
            pass

start=time.time()

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
stop=time.time()
print('执行时间：',stop-start) #2.664152145385742

start 0th
start 1th
start 2thstart 3th

start 4th
start 5th
start 6th
start 7th
start 8th
start 9th
finish 0th
finish 1th
finish 3thfinish 2th

finish 4th
finish 5th
finish 6th
finish 8th
finish 7th
finish 9th
执行时间： 1.0462768077850342


In [45]:
# 这种方法用线程实现requests异步爬虫代码如下
import concurrent.futures as cf
import asyncio
import requests
from bs4 import BeautifulSoup

start=time.time()
for i in range(10):
    get_title(i)
stop=time.time()
print('执行时间：',stop-start) # 9.7秒

def get_title(i):
    url = 'https://movie.douban.com/top250?start={}&filter='.format(i*25)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    # lis = soup.find('ol', class_='grid_view').find_all('li')
    # lis = soup.find('ol', class_='grid_view').find_all('li')
    # for li in lis:
    #     title = li.find('span', class_="title").text
    #     print(title)

async def main():
    with cf.ThreadPoolExecutor(max_workers = 10) as executor:
        loop = asyncio.get_event_loop()
        futures = (
            loop.run_in_executor(
                executor,
                get_title, 
                i)
            for i in range(10)
            )
        for result in await asyncio.gather(*futures):
            pass

start=time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
stop=time.time()
print('执行时间：',stop-start) # 1.2秒

执行时间： 7.96457314491272
执行时间： 2.283250570297241


In [59]:
# 单个线程的的异步爬虫，运用aiohttp
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import requests


def get_title_sync(i):
    url = 'https://movie.douban.com/top250?start={}&filter='.format(i*25)
    resp=requests.get(url)
    # print(resp.status)
    # text =  resp.text()
    print('start', i)

    # soup = BeautifulSoup(resp, 'html.parser')
    # lis = soup.find('ol', class_='grid_view').find_all('li')
    # for li in lis:
    #     title = li.find('span', class_="title").text
    #     print(title)

start=time.time()
for i in range(10):
    get_title_sync(i)
stop=time.time()
print('执行时间：',stop-start) # 8秒


async def get_title(i):
    url = 'https://movie.douban.com/top250?start={}&filter='.format(i*25)
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            print(resp.status)
            text = await resp.text()
            print('start', i)
    soup = BeautifulSoup(text, 'html.parser')
    lis = soup.find('ol', class_='grid_view').find_all('li')
    for li in lis:
        title = li.find('span', class_="title").text
        print(title)

start=time.time()
loop = asyncio.get_event_loop()
fun_list = (get_title(i) for i in range(10))
loop.run_until_complete(asyncio.gather(*fun_list))
stop=time.time()
print('执行时间：',stop-start) # 0.8秒

start 0
start 1
start 2
start 3
start 4
start 5
start 6
start 7
start 8
start 9
执行时间： 7.918631076812744
200
start 6
喜宴
英雄本色
谍影重重3
我是山姆
头号玩家
岁月神偷
哈利·波特与密室
一个叫欧维的男人决定去死
纵横四海
三块广告牌
疯狂原始人
心迷宫
达拉斯买家俱乐部
记忆碎片
卢旺达饭店
你的名字。
花样年华
萤火虫之墓
荒蛮故事
贫民窟的百万富翁
真爱至上
爆裂鼓手
东邪西毒
黑客帝国3：矩阵革命
未麻的部屋
200
start 4
7号房的礼物
阳光灿烂的日子
小森林 夏秋篇
第六感
红辣椒
重庆森林
入殓师
小森林 冬春篇
消失的爱人
爱在黎明破晓前
一一
唐伯虎点秋香
侧耳倾听
蝙蝠侠：黑暗骑士崛起
超脱
倩女幽魂
玛丽和马克思
无人知晓
超能陆战队
甜蜜蜜
萤火之森
告白
大鱼
阳光姐妹淘
驯龙高手
200
start 8
罗生门
终结者2：审判日
初恋这件小事
2001太空漫游
可可西里
魂断蓝桥
城市之光
海洋
牯岭街少年杀人事件
色，戒
燃情岁月
哈利·波特与火焰杯
穿越时空的少女
新龙门客栈
源代码
青蛇
无耻混蛋
九品芝麻官
遗愿清单
阿飞正传
完美陌生人
大佛普拉斯
血钻
谍影重重2
地球上的星星
200
start 2
大闹天宫
狮子王
死亡诗社
指环王2：双塔奇兵
猫鼠游戏
窃听风暴
钢琴家
指环王1：魔戒再现
黑客帝国
美丽心灵
饮食男女
两杆大烟枪
飞越疯人院
让子弹飞
本杰明·巴顿奇事
绿皮书
看不见的客人
西西里的美丽传说
拯救大兵瑞恩
海蒂和爷爷
小鞋子
穿条纹睡衣的男孩
情书
海豚湾
音乐之声
200
start 9
香水
疯狂的麦克斯4：狂暴之路
步履不停
战争之王
彗星来的那一夜
谍影重重
波西米亚狂想曲
猜火车
朗读者
浪潮
东京物语
再次出发之纽约遇见你
小萝莉的猴神大叔
驴得水
聚焦
追随
千钧一发
我爱你
一次别离
黑鹰坠落
黑客帝国2：重装上阵
发条橙
网络谜踪
四个春天
E.T. 外星人
200
start 0
肖申克的救赎
霸王别姬
阿甘正传
这个杀手不太冷
泰坦尼克号
美丽人生
千与千寻
辛德勒的名单
盗梦空间
忠犬八公的故事
海上钢琴师
星际穿越
楚门的世界
三傻大闹宝莱坞
机器人总